# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent


Create a simple Nutrition Assistant Agent

In [6]:
nutrition_agent = Agent(
    name="Dana Nutrition Agent",
    instructions="""
    You are a helpful assistant giving out nutirition advice.
    You give concise answers.
    """
)

Let's execute the Agent:

In [7]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Dana Nutrition Agent", ...)
- Final output (str):
    Bananas are a nutritious, convenient fruit. Key points:
    
    - Good for: potassium (helps blood pressure and fluid balance), vitamin B6, vitamin C, and fiber.
    - Energy: ~100 kcal per medium banana; includes natural sugars (moderate amount).
    - Fiber: supports digestion and helps with satiety.
    - Glycemic impact: moderate; ripe bananas have more sugar but still manageable as part of a balanced diet.
    - Cautions: high potassium can be an issue for some kidney conditions; check portions if you're watching sugar intake or if advised by a clinician.
    
    Bottom line: they’re a healthy, portable snack or add-on to meals when eaten in moderation.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [10]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Overall, bananas are healthy and convenient.

Key nutrients (per medium banana, ~105 calories):
- Potassium: supports heart and muscle function
- Vitamin B6
- Vitamin C
- Dietary fiber

Health benefits:
- Aids digestion and helps with satiety
- Provides quick energy from natural sugars
- Antioxidants and resistant starch (more in underripe bananas)

Things to consider:
- Moderate portions fit into most diets; about 1–2 per day is common
- Higher sugar content in very ripe bananas for those watching sugar intake
- Not suitable in excess for some kidney conditions (potassium limits)

Bottom line: a healthy, portable fruit that supports heart, digestion, and energy when eaten in moderation.

_Good Job!_